# Analyse der Tweets von Bundestagsabgeordneten

Training auf Basis von TRAIN_FILENAME und Vergabe der Labels.
Es wird itteriert über Wochen i Zeitraum Mai 2019 bis inkl. Juli 2020

## 1. Dataset preparation

In [2]:
import pickle, gensim, numpy as np
import pandas as pd 
import pymongo
import csv

client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client['Twitter']
All_Tweets_collection = db['twitter_mdp_ex_date_proj']

PICKLE_FOLDER_PATH = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/pickle_TU_Wien/'
Eval_Path = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Evaluierung/'
Path = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/'


# Bereits analysierte Tweets werden nicht erneut durchlaufen - lese Liste der pickle Files:
with open(PICKLE_FOLDER_PATH + 'Twitter_pickle.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        processed_weeks = row

In [3]:
import pandas as pd
import numpy as np
import pymongo
import datetime

pd.set_option('display.max_colwidth', 0)

# sample_df = pd.DataFrame(list(All_Tweets_collection.aggregate([ {"$sample": {"size": 50 }}], 
#                                      allowDiskUse=True
#                                    )))

time_sample_df = pd.DataFrame(list(All_Tweets_collection.find( {# 14.08.19, 06.11.19 fehlt 
            'created_at_datetime': {'$gte': datetime.datetime(2019,5,1,0,0,0), # 17.02. fehlt
                                    '$lt': datetime.datetime(2020,8,1,0,0,0)},#22
            'retweeted_id': None
            })
        ))# Load the regular expression library
pd.set_option('display.max_colwidth', 0)

import re

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F900-\U0001F9FF"  # https://www.compart.com/de/unicode/block/U+1F900
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

sample_df = time_sample_df
# Remove punctuation
sample_df.loc[:,('full_text_processed')] = sample_df.loc[:,('full_text')].map(lambda x: re.sub('[,\.!?#@\\n"“„\:;&\(\)]', '', x))
# Remove Links
sample_df.loc[:,('full_text_processed')] = sample_df.loc[:,('full_text_processed')].map(lambda x: re.sub('http.*', '', x))

sample_df.loc[:,('full_text_processed')] = sample_df.loc[:,('full_text_processed')].map(lambda x: re.sub('amp', '', x))
# Convert the titles to lowercase
#sample_df['full_text_processed'] = sample_df['full_text_processed'].map(lambda x: x.lower())

#sample_df['full_text_processed'] = sample_df['full_text_processed'].map(lambda x: remove_emoji(x))
# Print out the first rows of papers
#sample_df['full_text_processed'].head(200)

sample_df = sample_df[sample_df['full_text_processed'] != '']
sample_df = sample_df[sample_df['full_text_processed'] != ' ']
sample_df = sample_df[sample_df['full_text_processed'] != '  ']

sample_df.loc[:,('Monat')] = sample_df.created_at_datetime.dt.strftime('%y-%m')
sample_df.loc[:,('Woche')] = sample_df.created_at_datetime.dt.strftime('%y-w%U')
len(sample_df)

242894

In [1]:
#pd.set_option('display.max_rows', None)
#sample_df[sample_df['Woche'] == '20-w07'].sort_values('created_at_datetime')['created_at_datetime']
#print(sample_df['full_text_processed'])

# Features
Code von TUWien angepasst für die Anwendung auf eigene Tweets (TUWienKBS at GermEval 2018 : German Abusive Tweet Detection)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

import pickle, gensim, numpy as np

from utilities import get_train_data, get_test_data, Tokenizer, find_subtoken

#Test_TRAIN_FILENAME = Path + 'germEvalmore_train.txt' # Gelabelte Daten- random sample 3000 zum testen
#TRAIN_FILENAME = Path + 'germeval2018.train_imp.txt' # ALLE gelabelten Daten
TRAIN_FILENAME = Eval_Path + 'germeval2018.training_all_8541.txt' # Alle von GermEval gelabelten Daten
#TRAIN_FILENAME = Eval_Path + 'self_label_train_1000_und_8541.txt' # Alle von GermEval gelabelten + 1000 selbst gelabelte

#------------------------------
#source:
#http://www.cl.uni-heidelberg.de/english/research/downloads/resource_pages/GermanTwitterEmbeddings/GermanTwitterEmbeddings_data.shtml
MODEL_FILENAME  = "/home/lisa/Darmstadt/Master Arbeit/06_Analyse/twitter-de_d100_w5_min10.bin" # 821,8 MB
MODEL_DIMENSION = 100

In [4]:
X_train, y_train_t1, y_train_t2 = get_train_data(TRAIN_FILENAME)
len(X_train) #10343 # 7343 # 

8541

In [35]:
import pandas as pd
from utilities import get_train_data, get_test_data, Tokenizer, find_subtoken
test_rand = get_train_data(Eval_Path + 'self_label_test_802.txt')
print(len(test_rand[1]))
train = {'Text': test_rand[0], 'Label': test_rand[1], 'other_label': test_rand[2]} 
df_test_rand = pd.DataFrame(train)
#np.array(df_test_rand['Text'])

######## 20-w07, 19-w32, 19-w44, extra testen

802


In [36]:
# https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter
from datetime import datetime
processed_weeks = []

#for week in set(sample_df['Woche']):
#    print("%s: starting labeling" % (datetime.now().strftime("%H:%M:%S")))
#    print(week)
#    if week not in processed_weeks:
i=1
for i in range(0,2):
        #sample_mdp = np.array(sample_df[sample_df['Woche'] == week]['full_text_processed'])
        sample_mdp= np.array(df_test_rand['Text']) # Zur Evaluation der Ergebnisse/ des Alorithmus
        word2vec_model  = gensim.models.KeyedVectors.load_word2vec_format(MODEL_FILENAME, binary=True)

        X_train, y_train_t1, y_train_t2 = get_train_data(TRAIN_FILENAME)
        X_test_mdp                      = sample_mdp

    ### NGRAM FEATURES
    #  * Erstelle n-Gramme mit 3-7 Buchstaben (Funktion: char_vect)
    #  * Erstelle n-Gramme mit 1-3 Wörtern /Funkion: token_vect
    #  * Anwendung auf Training/ Test und mdp Daten

        char_vect  = TfidfVectorizer(analyzer="char", ngram_range=(3, 7), max_df=0.01, min_df=0.0002,
                                     preprocessor=Tokenizer(preserve_case=False, join=True).tokenize)

        token_vect = TfidfVectorizer(analyzer="word", ngram_range=(1, 3), max_df=0.01, min_df=0.0002,
                                     tokenizer=Tokenizer(preserve_case=False, use_stemmer=True).tokenize)

        X_CNGR_train = char_vect.fit_transform(X_train)
        X_CNGR_mdp = char_vect.transform(X_test_mdp)

        X_TNGR_train = token_vect.fit_transform(X_train)
        X_TNGR_mdp  = token_vect.transform(X_test_mdp)
        
        print('X_CNGR_mdp: ' + str(X_CNGR_mdp.shape))
        print('X_CNGR_train: ' + str(X_CNGR_train.shape))
        print('X_TNGR_mdp: ' + str(X_TNGR_mdp.shape))
        print('X_TNGR_train: ' + str(X_TNGR_train.shape))
        
        # X_CNGR_train = pickle.load(open(PICKLE_FOLDER_PATH + "X_CNGR_train.p", "rb" ))
        # X_TNGR_train = pickle.load(open(PICKLE_FOLDER_PATH + "X_TNGR_train.p", "rb" ))

        #pickle.dump(X_CNGR_train, open(PICKLE_FOLDER_PATH + "X_CNGR_train.p", "wb" ))
        #pickle.dump(X_TNGR_train, open(PICKLE_FOLDER_PATH + "X_TNGR_train.p", "wb" ))

    ### EMB FEATURES
    # * Tweets werden in Token unterteilt
    # * Prüfe ob die Token in einem Token im vortrainierten word2vec Model entsprechen
    # * Wenn nicht, teile Token in Präfix und Suffix und prüfe für diese das word2vec Model (ggf. beide in emb)
    # * emb enthält pro Tweet Vektoren für Token und wird normalisiert mit der Länge des Tweets + ggf extra Tokens
    # * X_EMB enthält die normalisierten Vektoren pro Tweet

        def get_EMB_feats(tweets):   
            tknzr = Tokenizer(preserve_case=True)
            tweets = [tknzr.tokenize(tweet) for tweet in tweets]

            X_EMB = []

            for tweet, i in zip(tweets, range(0,len(tweets))):
                emb = np.zeros(MODEL_DIMENSION)
                extra_tokens = 0

                for token in tweet:
                    try:
                        emb += word2vec_model[token]
                    except:
                        prefix = find_subtoken(token, word2vec_model, mode='initial')
                        suffix = find_subtoken(token, word2vec_model, mode='final')

                        if prefix != None and suffix != None:
                            emb += word2vec_model[prefix] + word2vec_model[suffix]
                            extra_tokens += 1
                        elif prefix != None and suffix == None:
                            emb += word2vec_model[prefix]
                        elif prefix == None and suffix != None:
                            emb += word2vec_model[suffix]           
                emb /= (len(tweet) + extra_tokens)
        #         print(i)
        #         print(len(tweet))
                X_EMB.append(emb)

            return normalize(X_EMB)
        
        # X_EMB_train = pickle.load(open(PICKLE_FOLDER_PATH + "X_EMB_train.p", "rb" ))        

        X_EMB_train = get_EMB_feats(X_train)
        X_EMB_mdp  = get_EMB_feats(X_test_mdp)
        
        print('X_EMB_mdp: ' + str(X_EMB_mdp.shape))
        print('X_EMB_train: ' + str(X_EMB_train.shape))
        pickle.dump(X_EMB_train, open(PICKLE_FOLDER_PATH + "X_EMB_train.p", "wb" ))

    # ### TIMP FEATURES
    # * Finden der wichtigen Tokens - also derer die in Tweets der angegebenen Kategorie verwendet werden
    # * Für diese wichtigsten Tokens werden die Features analog der EMB Features aus dem word2vec model abgeleitet
    # * Außerdem werden für alle Tweets analog der EMB feats abgeleitet
    # * Vergleiche mit der Cosine Similarity und gebe die höchsten und niedrigsten Werte pro Tweet zurück

        def k_most_imp_tokenlvl(k, category, max_df=0.01, min_df=0.0002):      
            token_vect = TfidfVectorizer(analyzer="word", ngram_range=(1, 1), lowercase=False,
                                         max_df=max_df, min_df=min_df,
                                         tokenizer=Tokenizer(preserve_case=True).tokenize)

            tfidf = token_vect.fit_transform(X_train)

            vocab = token_vect.vocabulary_
            inv_vocab = {index: word for word, index in vocab.items()}

            if category in ['OTHER', 'OFFENSE']:
                cat_ids = np.where(y_train_t1 == category)
            elif category in ['PROFANITY', 'ABUSE', 'INSULT']:
                cat_ids = np.where(y_train_t2 == category)

            most_imp_ids = np.argsort(np.asarray(np.mean(tfidf[cat_ids], axis=0)).flatten())[::-1]

            most_imp = []
            for index in most_imp_ids:
                most_imp.append(inv_vocab[index])

            return most_imp[:k]

        def get_TIMP_feats(tweets, k, category, max_df=0.01, min_df=0.0002):
            feats_max = []
            feats_min = []

            imp_tokens_vectors = []
            for imp_token in k_most_imp_tokenlvl(k, category, max_df=max_df, min_df=min_df):
                try:
                    imp_tokens_vectors.append(word2vec_model[imp_token])
                except:
                    imp_tokens_vectors.append(np.zeros(MODEL_DIMENSION))

            tknzr = Tokenizer(preserve_case=True)
            tweets = [tknzr.tokenize(tweet) for tweet in tweets]

            for tweet in tweets:
                tweet_vectors = []
                for token in tweet:
                    try:
                        tweet_vectors.append(word2vec_model[token])
                    except:
                        prefix = find_subtoken(token, word2vec_model, mode='initial')
                        suffix = find_subtoken(token, word2vec_model, mode='final')

                        if prefix != None and suffix != None:
                            tweet_vectors.append(word2vec_model[prefix])
                            tweet_vectors.append(word2vec_model[suffix])
                        elif prefix != None and suffix == None:
                            tweet_vectors.append(word2vec_model[prefix])
                        elif prefix == None and suffix != None:
                            tweet_vectors.append(word2vec_model[suffix])
                        else:
                            tweet_vectors.append(np.zeros(MODEL_DIMENSION))

                similarity = cosine_similarity(np.asarray(tweet_vectors), np.asarray(imp_tokens_vectors))

                feats_max.append(np.amax(similarity, axis=0))
                feats_min.append(np.amin(similarity, axis=0))

            return np.concatenate((feats_max, feats_min), axis=1)
        N_TIMP_TASK1 = 1250     

        #X_TIMP_task1_train = pickle.load(open(PICKLE_FOLDER_PATH + "X_TIMP_task1_train.p", "rb" ))

        X_TIMP_task1_train = \
        np.concatenate((get_TIMP_feats(X_train, N_TIMP_TASK1, 'OTHER'),
                        get_TIMP_feats(X_train, N_TIMP_TASK1, 'OFFENSE')), axis=1)


        X_TIMP_task1_mdp = \
        np.concatenate((get_TIMP_feats(X_test_mdp,  N_TIMP_TASK1, 'OTHER'),
                        get_TIMP_feats(X_test_mdp,  N_TIMP_TASK1, 'OFFENSE')), axis=1)
        
        print('X_TIMP_task1_mdp: ' + str(X_TIMP_task1_mdp.shape))
        print('X_TIMP_task1_train: ' + str(X_TIMP_task1_train.shape))
        pickle.dump(X_TIMP_task1_train, open(PICKLE_FOLDER_PATH + "X_TIMP_task1_train.p", "wb" ))

    ##### CIMP Features

        def k_most_imp_charlvl(k, category, max_df=0.01, min_df=0.0002):    
            char_vect  = TfidfVectorizer(analyzer="char", ngram_range=(3, 7), lowercase=False,
                                         max_df=max_df, min_df=min_df,
                                         preprocessor=Tokenizer(preserve_case=True, join=True).tokenize)

            tfidf = char_vect.fit_transform(X_train)

            vocab = char_vect.vocabulary_
            inv_vocab = {index: word for word, index in vocab.items()}

            if category in ['OTHER', 'OFFENSE']:
                cat_ids = np.where(y_train_t1 == category)
            elif category in ['PROFANITY', 'ABUSE', 'INSULT']:
                cat_ids = np.where(y_train_t2 == category)       

            most_imp_ids = np.argsort(np.asarray(np.mean(tfidf[cat_ids], axis=0)).flatten())[::-1]

            most_imp = []
            for index in most_imp_ids:
                most_imp.append(inv_vocab[index])

            return most_imp[:k]

        def get_CIMP_feats(tweets, k, category, max_df=0.01, min_df=0.0002):
            feats = np.zeros((len(tweets), k))
            for imp_ngram_index, imp_ngram in enumerate(k_most_imp_charlvl(k, category, max_df=max_df, min_df=min_df)):
                for tweet_index, tweet in enumerate(tweets):
                    if tweet.find(imp_ngram) != -1:
                        feats[tweet_index][imp_ngram_index] = 1
            return feats

        N_CIMP_TASK1 = 3200
        N_CIMP_TASK2 = 370
        
        # X_CIMP_task1_train = pickle.load(open(PICKLE_FOLDER_PATH + "X_CIMP_task1_train.p", "rb" ))

        X_CIMP_task1_train = \
        np.concatenate((get_CIMP_feats(X_train, N_CIMP_TASK1, 'OTHER'),
                        get_CIMP_feats(X_train, N_CIMP_TASK1, 'OFFENSE')), axis=1)


        X_CIMP_task1_mdp = \
        np.concatenate((get_CIMP_feats(X_test_mdp,  N_CIMP_TASK1, 'OTHER'),
                        get_CIMP_feats(X_test_mdp,  N_CIMP_TASK1, 'OFFENSE')), axis=1)
        
        print('X_CIMP_task1_mdp: ' + str(X_CIMP_task1_mdp.shape))
        print('X_CIMP_task1_train: ' + str(X_CIMP_task1_train.shape))
        pickle.dump(X_CIMP_task1_train, open(PICKLE_FOLDER_PATH + "X_CIMP_task1_train.p", "wb" ))
        
    #### Predictions
        _, y1, y2 = get_train_data(TRAIN_FILENAME)
        


    ## Funktion für das Aufteilen in Train und Test Sample 
    # -> StratifiedKFold sorgt dafür, dass das prozentuale Verhältnis der Klassen im jeweiligen Sample (Test, Train) gleich ist


        from sklearn.ensemble import ExtraTreesClassifier
        from sklearn.linear_model import LogisticRegression

        from sklearn.model_selection import cross_val_predict, StratifiedKFold

        def get_META_feats(clf, X_train, mdp, y, seeds=[42]):
            feats_train = []
            for seed in seeds:
                skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
                feats_train.append(cross_val_predict(clf, X_train, y=y, method='predict_proba', cv=skf, n_jobs=-1))
            feats_train = np.mean(feats_train, axis=0)
            print('Länge bzw Anzahl an Features_train: ' + str(len(feats_train)))
            print('Classifier: ' + str(clf))
            clf.fit(X_train, y)
            feats_mdp = clf.predict_proba(mdp)
            print('Länge bzw Anzahl an Features_mdp: ' + str(len(feats_mdp)))

            return feats_train, feats_mdp

    # ## TASK 1 - Base level predictions
    # Die drei verschiedenen Classifier (clfs_task1) werden auf die Feature Vectoren (base_feats_task1) angewandt.
    # Von einer 10-fold CrossVal wird für den Trainings Feature Satz der Durchschnitt genommen (jeder Spalte).
    # Bei den Test-/mdp Daten wird keine Cross Val durchgeführt (keine y Variablen) sondern nur mit jedem Classifier eine prediction anhand der Feature Vektoren gemacht


        clfs_task1 = [LogisticRegression(class_weight='balanced', max_iter=200),
                      ExtraTreesClassifier(n_estimators=100, criterion='entropy', n_jobs=-1),
                      ExtraTreesClassifier(n_estimators=100, criterion='gini', n_jobs=-1)]

        base_feats_task1 = [(X_CIMP_task1_train, X_CIMP_task1_mdp),
                            (X_TIMP_task1_train, X_TIMP_task1_mdp),
                            (X_CNGR_train, X_CNGR_mdp),
                            (X_TNGR_train, X_TNGR_mdp),
                            (X_EMB_train, X_EMB_mdp)]
        X_META_task1_train = []
        #X_META_task1_test  = []
        X_META_task1_mdp  = []
        for X_train, mdp in base_feats_task1:                 # X-train z.B X_CIMP_task1_train, mdp z.B X_CIMP_task1_mdp
            for clf in clfs_task1:
                feats = get_META_feats(clf, X_train, mdp, y1)
                print(str(X_train)+str(clf)+': ' + str(len(feats[0])))
                print(str(mdp)+str(clf)+': ' + str(len(feats[1])))                

                X_META_task1_train.append(feats[0])           # aus "get_META_feats: feats_train
                X_META_task1_mdp.append(feats[1])             # aus "get_META_feats: feats_mdp

        X_META_task1_train = np.concatenate(X_META_task1_train, axis=1)
        X_META_task1_mdp  = np.concatenate(X_META_task1_mdp, axis=1)
        print('train: '+str(X_META_task1_train.shape))
        print('mdp: '+str(X_META_task1_mdp.shape))

        clf_task1 = LogisticRegression(C=0.17, class_weight='balanced', max_iter=1000)
        clf_task1.fit(X_META_task1_train, y1) 

        preds_task1 = clf_task1.predict(X_META_task1_mdp) 
        
        save_df = df_test_rand
        save_df['predict'] = preds_task1
        pickle.dump(save_df, open(Eval_Path + 'Eval_1.p', "wb" ))

        #pd.set_option('display.max_colwidth', 0)
        #save_df = sample_df[sample_df['Woche'] == week]
        #save_df['predict'] = preds_task1
        #pickle.dump(save_df, open(PICKLE_FOLDER_PATH + 'Twitter_Sent_Wien_'+week+ '_3.p', "wb" ))
        #print("%s: end labeling" % (datetime.now().strftime("%H:%M:%S")))
        
    #else:
    #    print("Wiederholung: " + week + " wurde bereits verwendet")



X_CNGR_mdp: (802, 286641)
X_CNGR_train: (8541, 286641)
X_TNGR_mdp: (802, 31082)
X_TNGR_train: (8541, 31082)
X_EMB_mdp: (802, 100)
X_EMB_train: (8541, 100)
X_TIMP_task1_mdp: (802, 5000)
X_TIMP_task1_train: (8541, 5000)
X_CIMP_task1_mdp: (802, 6400)
X_CIMP_task1_train: (8541, 6400)
Länge bzw Anzahl an Features_train: 8541
Classifier: LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=200, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Länge bzw Anzahl an Features_mdp: 802
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
   

/usr/local/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Länge bzw Anzahl an Features_train: 8541
Classifier: ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='entropy', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
Länge bzw Anzahl an Features_mdp: 802
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='entropy', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
          

/usr/local/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Länge bzw Anzahl an Features_train: 8541
Classifier: ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
Länge bzw Anzahl an Features_mdp: 802
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                

...

Länge bzw Anzahl an Features_mdp: 802
[[ 0.51321657  0.60198338  0.35097487 ... -0.0416928   0.04020016
  -0.035798  ]
 [ 0.37136491  0.55409823  0.51642371 ... -0.01057631 -0.1064922
  -0.02542487]
 [ 0.42756603  0.25808367  0.40140532 ... -0.12117179 -0.07554074
  -0.07992783]
 ...
 [ 0.40381768  0.68656824  0.55551403 ...  0.00638522  0.08741061
   0.02864218]
 [ 0.20871208  0.58225346  0.5254333  ... -0.05331408 -0.04771642
   0.07319433]
 [ 0.46545498  0.72501504  0.44013603 ...  0.01042647 -0.0127348
  -0.02542487]]LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=200, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False): 8541
[[ 3.66580886e-01  6.97863735e-01  7.82714377e-01 ... -8.29838347e-02
  -7.67481688e-03 -5.32745857e-02]
 [ 3.93140909e-01  4.87555296e

Länge bzw Anzahl an Features_train: 8541
Classifier: ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='entropy', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
Länge bzw Anzahl an Features_mdp: 802
  (0, 118064)	0.06832140888518141
  (0, 85771)	0.07235548395633176
  (0, 56560)	0.07235548395633176
  (0, 6845)	0.07235548395633176
  (0, 259821)	0.07991781952198222
  (0, 246494)	0.07535857894903215
  (0, 242057)	0.07373131555349188
  (0, 226599)	0.06497718044864063
  (0, 28501)	0.07535857894903215
  (0, 190313)	0.07991781952198222
  (0, 195588)	0.071163684417131
  (0, 148

Länge bzw Anzahl an Features_train: 8541
Classifier: LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=200, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Länge bzw Anzahl an Features_mdp: 802
  (0, 27994)	0.3838999798572487
  (0, 167)	0.3838999798572487
  (0, 11815)	0.2750439403096284
  (0, 12577)	0.37156589032095133
  (0, 29225)	0.3541819674903452
  (0, 27993)	0.336798044659739
  (0, 12836)	0.309530643789222
  (0, 19704)	0.309530643789222
  (0, 12574)	0.24705263226499857
  (1, 820)	0.2586172666205532
  (1, 3661)	0.2586172666205532
  (1, 13449)	0.243863381418298
  (1, 29037)	0.23028853460577484
  (1, 5921)	0.243863381418298
  (1, 6299)	0.2141254709135135
  (1, 819)	0.22688667435053542
  (1, 26623)	0.23859749185895449
  (1, 1352)	0.21626588921451245
  (1, 2216

Länge bzw Anzahl an Features_train: 8541
Classifier: ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
Länge bzw Anzahl an Features_mdp: 802
  (0, 27994)	0.3838999798572487
  (0, 167)	0.3838999798572487
  (0, 11815)	0.2750439403096284
  (0, 12577)	0.37156589032095133
  (0, 29225)	0.3541819674903452
  (0, 27993)	0.336798044659739
  (0, 12836)	0.309530643789222
  (0, 19704)	0.309530643789222
  (0, 12574)	0.24705263226499857
  (1, 820)	0.2586172666205532
  (1, 3661)	0.2586172666205532
  (1, 13449)	0.243863381418298
  

Länge bzw Anzahl an Features_train: 8541
Classifier: ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
Länge bzw Anzahl an Features_mdp: 802
[[-0.11466956 -0.01908347  0.00172211 ...  0.06156928  0.08135319
  -0.07859743]
 [-0.12918881 -0.02765546  0.11615171 ... -0.00593668 -0.0802549
  -0.00568726]
 [-0.04746682 -0.10253195 -0.02464943 ...  0.13990997 -0.00122685
   0.00821853]
 ...
 [-0.08659905 -0.00710907  0.0022457  ...  0.01209016  0.03789412
  -0.08774336]
 [-0.17021359  0.03429338 -0.02271174 ...  0.043111

In [32]:
X_META_task1_train.shape

(9541, 30)

In [19]:
save_df = df_test_rand
save_df['predict'] = preds_task1
pickle.dump(save_df, open(Eval_Path + 'Eval_1.p', "wb" ))